Para empezar a utilizar mongodb desde 0, vamos a partir realizando los siguientes pasos:


1.   Instalaremos $\texttt{mongodb}$
2.   Inicializaremos el proceso $\texttt{mongodb}$
3.   Instalaremos el comando $\texttt{gdown}$ para bajar la base de datos en formato json de google drive.
4.   Importaremos los datos por linea de comando. Dejaremos los datos en la base de datos $\texttt{local}$, en la colección $\texttt{pokedex}$, y le informaremos al comando que los datos vienen como una arreglo de jsons.




In [ ]:
!apt install mongodb
!service mongodb start
!pip install gdown
!pip install pymongo
!gdown https://drive.google.com/uc?id=108aLE4mFE5bZYitFfX5c14Qj_aYA5MO_
!mongoimport --db local --collection pokedex --jsonArray < pokedex.json

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpcap0.8 libyaml-cpp0.6 mongo-tools mongodb-clients mongodb-server
  mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libyaml-cpp0.6 mongo-tools mongodb mongodb-clients mongodb-server
  mongodb-server-core
0 upgraded, 7 newly installed, 0 to remove and 38 not upgraded.
Need to get 55.8 MB of archives.
After this operation, 226 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libpcap0.8 amd64 1.9.1-3 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libyaml-cpp0.6 amd64 0.6.2-4ubuntu1 [124 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 mongo-tools amd64 3.6.3-0ubuntu1 [12.3 MB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 mongodb-clients amd64 1:3.6.9+really3.6.8+90~g8e540c0b6d-0ubuntu5.3 [21.6 MB]
Get:5 

A continuación importaremos la librería de mongo para python, crearemos un cliente que se conectara a $\texttt{localhost}$ por defecto, nos conectaremos a la base local, y luego a la colección pokedex la cual almacenaremos en la variable $\texttt{collection}$.

In [ ]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']
db = client['local']
collection = db['pokedex']

Para partir, podemos listar toda la colección usando el comando $\texttt{collection.find({})}$. Para no saturar la salida, nos limitaremos a mirar el primero de los registros.

In [ ]:
import pprint
for pokemon in collection.find({}).limit(1):
    pprint.pprint(pokemon)

{'Abilities': ['Overgrow', 'Chlorophyll'],
 'Attack': 49,
 'Capture Rate': 45,
 'Category': 'Seed Pokémon',
 'Defense': 49,
 'Egg Steps': 5120,
 'Exp Group': 'Medium Slow',
 'HP': 45,
 'Height (ft)': '2\'04"',
 'Height (m)': 0.7,
 'Id': '001',
 'Moves': {'Double-Edge': {'Accuracy': 100,
                           'Description': 'A reckless, life-risking tackle. '
                                          'This also damages the user quite a '
                                          'lot.',
                           'Effect %': '--',
                           'Level': 27,
                           'PP': 15,
                           'Power': 120,
                           'Type': 'Normal'},
           'Growl': {'Accuracy': 100,
                     'Description': 'The user growls in an endearing way, '
                                    'making opposing Pokémon less wary. This '
                                    'lowers their Attack stat.',
                     'Effect %': '--'

Fijense que aparte de los datos básicos del pokemon, también se presentan las habilidades (una lista de strings), y los movimientos (un arreglo asociativo/diccionario de los movimientos y sus detalles).

Si queremos ahora listar los pokemones que se llaman $\texttt{Charmander}$ podemos hacer la siguiente consulta:

In [ ]:
for pokemon in collection.find({'Name': 'Charmander'}):
    pprint.pprint(pokemon)

{'Abilities': ['Blaze', 'Solar Power'],
 'Attack': 52,
 'Capture Rate': 45,
 'Category': 'Lizard Pokémon',
 'Defense': 43,
 'Egg Steps': 5120,
 'Exp Group': 'Medium Slow',
 'HP': 39,
 'Height (ft)': '2\'00"',
 'Height (m)': 0.6,
 'Id': '004',
 'Moves': {'Dragon Rage': {'Accuracy': 100,
                           'Description': 'This attack hits the target with a '
                                          'shock wave of pure rage. This '
                                          'attack always inflicts 40 HP '
                                          'damage.',
                           'Effect %': '--',
                           'Level': 16,
                           'PP': 10,
                           'Power': '??',
                           'Type': 'Dragon'},
           'Ember': {'Accuracy': 100,
                     'Description': 'The target is attacked with small flames. '
                                    'This may also leave the target with a '
                         

Y si queremos proyectar algunos de sus atributos lo hacemos tal como si fuera un diccionario:

In [ ]:
for pokemon in collection.find({'Name': 'Mew'}):
    print(pokemon['Speed'],pokemon['Abilities'])

100 ['Synchronize']


Recomendamos hechar un vistazo [aquí](https://docs.mongodb.com/manual/tutorial/query-documents/) para ver otros ejemplos de consultas que utilizarán en las siguientes preguntas:

#Pregunta 1
Encuentre los nombre de los pokemons que pesen mas de 150kg.
[Ordene](https://www.w3schools.com/python/python_mongodb_sort.asp) los resultados por peso descendiente

In [ ]:
for pokemon in collection.find({'Weight (kg)': {'$gt': 150}}).sort('Weight (kg)', -1):
    pprint.pprint(pokemon['Name'])

#Pregunta 2
Encuentre los nombres de los pokemones ''Glass cannon'', que son los que tienen ataque sobre 120 y defensa bajo 50.

In [ ]:
for pokemon in collection.find({'Attack': {'$gt': 120}, 'Defense': {'$lt': 50}}):
    pprint.pprint(pokemon['Name'])

'Cranidos'
'Pheromosa'


#Pregunta 3
Encuentre los pokemones "defensivos", que son los que tienen ya sea defensa, hp, o defensa especial (sp. defense) mayor o igual a 120.

In [ ]:
for pokemon in collection.find({'$or': [{'Defense': {'$gte': 120}},{'Hp': {'$gte': 120}}, {'Sp. Defense': {'$gte': 120}}]}):
    pprint.pprint(pokemon['Name'])

#Pregunta 4
Seleccione los pokemones que tienen el movimiento $\texttt{Take Down}$ de nivel 15.

**Hint**: Puede consultar en diccionarios embebidos concatenando las llaves con un punto ".".

Por ejemplo, para un dato:
```
[{
  a: {
    b: 'foo'
  }
}]
```
se puede consultar por los documentos tal que $\texttt{b}$ sea $\texttt{foo}$, escribiendo $\texttt{collection.find({'a.b': 'foo'})}$.

In [ ]:
for pokemon in collection.find({'Moves.Take Down.Level': 15}):
    pprint.pprint(pokemon['Name'])

'Bulbasaur'
'Ivysaur'
'Venusaur'
'Cranidos'
'Rampardos'
'Shieldon'
'Bastiodon'
'Lillipup'
'Herdier'
'Stoutland'
'Bunnelby'
'Diggersby'
'Aurorus'
'Amaura'
'Bergmite'
'Avalugg'


#Pregunta 5
Seleccione los pokemones que tienen el movimiento $\texttt{Take Down}$.

**Hint**: Puede testear si es que una llave existe utilizando [$exists](https://docs.mongodb.com/manual/reference/operator/query/exists/)

In [ ]:
for pokemon in collection.find({'Moves.Take Down': { '$exists': True }}):
    pprint.pprint(pokemon['Name'])

'Bulbasaur'
'Ivysaur'
'Venusaur'
'Growlithe'
'Ponyta'
'Rapidash'
'Seel'
'Dewgong'
'Rhyhorn'
'Rhydon'
'Chansey'
'Tauros'
'Eevee'
'Aerodactyl'
'Hoothoot'
'Noctowl'
'Chinchou'
'Lanturn'
'Mareep'
'Flaaffy'
'Ampharos'
'Pineco'
'Forretress'
'Dunsparce'
'Qwilfish'
'Heracross'
'Swinub'
'Piloswine'
'Mantine'
'Phanpy'
'Stantler'
'Blissey'
'Mudkip'
'Swampert'
'Marshtomp'
'Poochyena'
'Zigzagoon'
'Mightyena'
'Aron'
'Lairon'
'Aggron'
'Carvanha'
'Numel'
'Camerupt'
'Swablu'
'Altaria'
'Chimecho'
'Relicanth'
'Luvdisc'
'Beldum'
'Metang'
'Metagross'
'Starly'
'Staravia'
'Staraptor'
'Bidoof'
'Bibarel'
'Cranidos'
'Rampardos'
'Shieldon'
'Bastiodon'
'Cherubi'
'Cherrim'
'Gible'
'Gabite'
'Garchomp'
'Hippopotas'
'Hippowdon'
'Mantyke'
'Rhyperior'
'Mamoswine'
'Tepig'
'Emboar'
'Pignite'
'Lillipup'
'Herdier'
'Stoutland'
'Audino'
'Basculin'
'Trubbish'
'Sawsbuck'
'Karrablast'
'Deerling'
'Larvesta'
'Terrakion'
'Cobalion'
'Virizion'
'Keldeo'
'Chespin'
'Quilladin'
'Chesnaught'
'Bunnelby'
'Diggersby'
'Litleo'
'Pyroar'
'Ski

#Pregunta 6
1. Seleccione los pokemones que tienen la habilidad $\texttt{Telepathy}$


In [ ]:
for pokemon in collection.find({'Abilities': 'Telepathy'}):
    pprint.pprint(pokemon['Name'])

'Wobbuffet'
'Ralts'
'Kirlia'
'Gardevoir'
'Meditite'
'Medicham'
'Wynaut'
'Dialga'
'Palkia'
'Giratina'
'Musharna'
'Munna'
'Elgyem'
'Beheeyem'
'Noibat'
'Noivern'
'Oranguru'
'Tapu Koko'
'Tapu Lele'
'Tapu Bulu'
'Tapu Fini'


2. Seleccione los pokemones que tienen la habilidad $\texttt{Telepathy}$
 **y** $\texttt{Synchronize}$ (pueden tener mas habilidades aparte de estas dos.

   **Hint**: puede utilizar el operador [$all](https://docs.mongodb.com/manual/tutorial/query-arrays/) .

In [ ]:
for pokemon in collection.find({'Abilities': {'$all' : ['Telepathy','Synchronize']}}):
    pprint.pprint(pokemon['Name'])

'Ralts'
'Kirlia'
'Gardevoir'
'Musharna'
'Munna'
'Elgyem'
'Beheeyem'


#Pregunta 7
Para la siguiente pregunta vamos a crear un índice sobre la columna $\texttt{Category}$.

In [ ]:
collection.create_index([('Category', 'text')])

'Category_text'

Usando el operador [$text](https://docs.mongodb.com/manual/text-search/) responda a las siguientes consultas.
1.  Devuelva los nombres y categorías de los pokemons tales que su categoría contenga ya sea la palabra **Sea** o **Gas**.

In [ ]:
for pokemon in collection.find({'$text': {'$search': 'Sea Gas'}}):
    print(pokemon['Name'], ":", pokemon['Category'])


Haunter : Gas Pokémon
Gastly : Gas Pokémon
Weezing : Poison Gas Pokémon
Koffing : Poison Gas Pokémon
Dhelmise : Sea Creeper Pokémon
Pyukumuku : Sea Cucumber Pokémon
Popplio : Sea Lion Pokémon
Oshawott : Sea Otter Pokémon
Phione : Sea Drifter Pokémon
Gastrodon : Sea Slug Pokémon
Shellos : Sea Slug Pokémon
Buizel : Sea Weasel Pokémon
Floatzel : Sea Weasel Pokémon
Kyogre : Sea Basin Pokémon
Gorebyss : South Sea Pokémon
Huntail : Deep Sea Pokémon
Lileep : Sea Lily Pokémon
Dewgong : Sea Lion Pokémon
Seel : Sea Lion Pokémon


2. Devuelva los nombres y categorías de los pokemons tales que su categoría contenga la palabra **Sea** y no **Deep**.

In [ ]:
for pokemon in collection.find({'$text': {'$search': 'Sea -Deep'}}):
    print(pokemon['Name'], ":", pokemon['Category'])


Dhelmise : Sea Creeper Pokémon
Pyukumuku : Sea Cucumber Pokémon
Popplio : Sea Lion Pokémon
Oshawott : Sea Otter Pokémon
Phione : Sea Drifter Pokémon
Gastrodon : Sea Slug Pokémon
Shellos : Sea Slug Pokémon
Buizel : Sea Weasel Pokémon
Floatzel : Sea Weasel Pokémon
Kyogre : Sea Basin Pokémon
Gorebyss : South Sea Pokémon
Lileep : Sea Lily Pokémon
Dewgong : Sea Lion Pokémon
Seel : Sea Lion Pokémon


3. Devuelva los nombres y categorías de los pokemons tales que su categoría contenga ambas palabras **Sea** y **Lion**.

In [ ]:
for pokemon in collection.find({'$text': {'$search': '\"Sea\" \"Lion\"'}}):
    print(pokemon['Name'], ":", pokemon['Category'])


Popplio : Sea Lion Pokémon
Dewgong : Sea Lion Pokémon
Seel : Sea Lion Pokémon
